In [1]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI

from tqdm import tqdm
tqdm.pandas()

import datetime

from ast import literal_eval

# Article Data Loading 

In [31]:
art_data = pd.read_csv('../../data/webpage_data/full_flipside_data.csv')

In [32]:
art_data.head()

,article_url,date,title,left_sum,right_sum,linked_arts
0,https:/theflipside.io/archives/checks-in-the-mail,"March 20, 2020",Checks In The Mail,The left supports cash payments and argues tha...,The right is generally supportive of helping c...,['https://apnews.com/0373a78b0aacc49d1f6bd8901...
1,https:/theflipside.io/archives/eos-regarding-t...,"January 26, 2021",EOs Regarding Transgender Rights,The left is supportive of both policies.,The right is critical of both policies.,['https://apnews.com/article/biden-reverse-ban...
2,https:/theflipside.io/archives/facebook-and-br...,"April 2, 2019",Facebook and Breaking Up Big Tech,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...,['https://www.washingtonpost.com/opinions/mark...
3,https:/theflipside.io/archives/general-electio...,"October 28, 2020",General Election Update,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,['https://theflipside.us15.list-manage.com/tra...
4,https:/theflipside.io/archives/impeachment-hea...,"November 22, 2019",Impeachment Hearings Continue,"The left supports impeachment, arguing that th...","The right opposes impeachment, arguing that th...",['https://theflipside.us15.list-manage.com/tra...


In [33]:
art_data['linked_arts'] = art_data['linked_arts'].str.replace('\n', ',')
art_data['linked_arts'] = art_data['linked_arts'].apply(literal_eval)

In [34]:
both_queries = art_data[['title', 'linked_arts', 'date']]
both_queries = both_queries.values

In [35]:
title_queries = [[query[0], query[2]] for query in both_queries]
url_queries = [[query[1], query[2]] for query in both_queries]

In [36]:
len(both_queries)

982

# Reddit Scraping 

In [9]:
api = PushshiftAPI()

In [13]:
gen = api.search_submissions(subreddit = 'politics', q = 'trump')

In [14]:
MAX_COLS = 74 # Maximum number of columns returned for submissions
while True:
    data = next(gen).d_
    if len(data) >= MAX_COLS:
        break

In [15]:
COLUMNS = list(data.keys())

In [45]:
def get_posts_by_news(query, output_csv, title_or_url = 'query', subreddit = 'politics', date_thresh = 10, header = True, max_posts = 10, retry = True):
    title = query[0]
    url = query[1]
    date = datetime.datetime.strptime(query[2], '%B %d, %Y')
    
    if date_thresh:
        min_date = int((date - datetime.timedelta(days = date_thresh)).timestamp())
        max_date = int((date + datetime.timedelta(days = date_thresh)).timestamp())
    else:
        min_date = None
        max_date = None
        
    if title_or_url == 'title':
        gen = api.search_submissions(subreddit = subreddit, q = title, after = min_date, before = max_date)
    elif title_or_url == 'url':
        gen = api.search_submissions(subreddit = subreddit, url = url, after = min_date, before = max_date)
    else:
        print('Must specify title or url')
        return
    
    num_collected = 0
    for post in gen:
        data = post.d_
        data_df = pd.DataFrame({key:str(val) for key, val in data.items()}, index = [num_collected], columns = COLUMNS).fillna('None')
        data_df['title_q'] = title
        data_df['date_q'] = date
        
        data_df.to_csv(output_csv, mode = 'a', escapechar = '\\', header = header)
        num_collected += 1
        
        if num_collected > max_posts:
            break
    
    if num_collected == 0:
        print(f'Query {query} returned 0 results. {"Retrying with less terms" if retry and title_or_url == "title" else ""}')
        if retry and title_or_url == 'title':
            new_query = [' '.join(title.split(' ')[:2]), query[1], query[2]]
            get_posts_by_news(new_query, output_csv, title_or_url = title_or_url, subreddit = subreddit, date_thresh = date_thresh, header = header, max_posts = max_posts, retry = False)
    else:
        print(f'Query {query} returned {num_collected} results.')

In [225]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(queries):
    get_posts_by_news(query, OUTPUT_CSV, header = WRITE_HEAD)
    print(f'Finished query {i} of {len(queries)}', end = '\r')

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)


In [226]:
reddit_posts = pd.read_csv(OUTPUT_CSV)

C:\Users\ndeas\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (2,27,30,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [235]:
reddit_posts.groupby('title_q').count().sort_values('author', ascending = False).head(50)

,Unnamed: 0,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,created,date_q
title_q,,,,,,,,,,,,,,,,,,,,,
Ukraine,7534,7534,7534,7534,7534,7534,7534,7534,7525,7534,...,7534,7534,7534,7534,7522,7522,7534,7534,7534,7534
Coronavirus,7320,7320,7320,7320,7320,7320,7320,7320,7316,7320,...,7320,7320,7320,7320,7320,7320,7320,7320,7320,7320
Impeachment,5904,5904,5904,5904,5904,5904,5904,5904,5894,5904,...,5904,5904,5904,5904,5904,5904,5904,5904,5904,5904
Kavanaugh,2110,2110,2110,2110,2110,2110,2110,2110,2110,2110,...,2110,2110,2110,2110,2110,2110,2110,2110,2110,2110
Hunter Biden,980,980,980,980,980,980,980,980,979,980,...,980,980,980,980,980,980,980,980,980,980
Joe Biden,916,916,916,916,916,916,916,916,915,916,...,916,916,916,916,916,916,916,916,916,916
State of the Union,912,912,912,912,912,912,912,912,911,912,...,912,912,912,912,912,912,912,912,912,912
Portland,837,837,837,837,837,837,837,837,837,837,...,837,837,837,837,837,837,837,837,837,837
Kenosha,676,676,676,676,676,676,676,676,676,676,...,676,676,676,676,676,676,676,676,676,676


In [236]:
reddit_posts.columns

Index(['Unnamed: 0', 'all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_is_blocked', 'author_patreon_flair', 'author_premium',
       'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain',
       'full_link', 'gildings', 'id', 'is_created_from_ads_ui',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_text', 'link_flair_text_color',
       'link_flair_type', 'locked', 'media', 'media_embed', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
       'preview', 'pwls', 'removed_by_category'

In [240]:
reddit_short = reddit_posts[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

In [242]:
NUM_SAMPLE = 5
samples = []
for i, group in reddit_short.groupby('upvote_ratio'):
    group = group.sort_values('upvote_ratio', ascending = False)
    new_group = group.iloc[:NUM_SAMPLE]
    samples.append(new_group)

In [244]:
reddit_short = pd.concat(samples, axis = 0)

In [245]:
reddit_short

,title_q,date_q,title,selftext,upvote_ratio
37784,Impeachment,2021-01-12,"To move on, drop the Trump impeachment trial",NaN,0.01
7953,Hunter Biden,2020-12-11,Hunter Biden emailed best wishes from the ‘ent...,[deleted],0.02
7960,Hunter Biden,2020-12-11,"Hunter Biden Email Reportedly Shows $400,000 i...",NaN,0.03
17695,Michael Flynn,2020-06-25,Sidney Powell: Michael Flynn Planned to ‘Audit...,NaN,0.03
38863,Impeachment,2021-01-12,Trump faces impeachment again as Democrats mak...,NaN,0.03
...,...,...,...,...,...
0,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
39561,Impeachment,2020-01-16,Nadler Says Trump Acting Like ‘a Dictator’: Im...,NaN,None
39570,Impeachment,2020-01-16,The impeachment evidence will catch up to Repu...,NaN,None
39569,Impeachment,2020-01-16,Democrats feel political momentum swinging to ...,NaN,None


In [246]:
titles = reddit_short['title_q'].unique()

In [247]:
titles

array(['Impeachment', 'Hunter Biden', 'Michael Flynn',
       'Biden-Putin Summit', 'Merrick Garland', 'Statues', 'Debt Ceiling',
       'Obamagate', 'Dr. Seuss', 'Minimum Wage',
       'Climate Executive Orders', 'Neera Tanden', 'Portland',
       'Biden’s Speech', 'Marjorie Taylor Greene', 'Airstrikes in Syria',
       'Critical Race Theory', 'HEROES Act',
       'Supreme Court on Religious Schools', 'Amy Coney Barrett',
       'Biden’s Inauguration', 'Equality Act', 'Infrastructure Plan',
       'Immigration', 'Police Protests', 'Jeff Sessions', 'GameStop',
       'Police Reform', 'Jobs Report', 'Ukraine', 'Ketanji Brown Jackson',
       'Russia-Taliban Bounties', 'Education Commission',
       'Trump Signs Executive Order', 'COVID Vaccine',
       'Afghanistan and Biden', 'COVID Stimulus',
       'Immigration Restrictions', 'Pete Buttigieg', 'Elon Musk',
       'Bob Dole', 'Bahrain-Israel Deal', 'Wildfires', 'Andrew Cuomo',
       'Juneteenth', 'COVID Relief Bill', 'G7 Summit', 'Ah

In [298]:
len(np.unique(src_titles))

851

In [252]:
miss_titles = set(src_titles).difference(titles)

In [284]:
miss_queries = [list(query) for query in queries if query[0] in miss_titles]

In [289]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(miss_queries):
    get_posts_by_news(query, OUTPUT_CSV, date_thresh = 30, header = WRITE_HEAD, max_posts = 10)
    print(f'Finished query {i} of {len(miss_queries)}', end = '\r')

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [291]:
reddit_posts2 = pd.read_csv(OUTPUT_CSV)

C:\Users\ndeas\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (27,30,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [316]:
reddit_short2 = reddit_posts2[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

In [317]:
reddit_short2

,title_q,date_q,title,selftext,upvote_ratio
0,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
1,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
2,Impeachment Hearings Continue,2019-11-22,"Hyper-Partisan Witnesses, Democrats Continue E...",NaN,None
3,Inflation,2022-01-25,Every politician that did extended lockdowns n...,NaN,1.0
4,Inflation,2022-01-25,AOC: Corporate 'price gouging' is fueling infl...,NaN,1.0
...,...,...,...,...,...
62387,Afghanistan and the World,2021-08-19,Australian view on the withdrawal from Afghani...,NaN,1.0
62388,Afghanistan and the World,2021-08-19,How Joe Biden failed the people of Afghanistan...,NaN,1.0
62389,Afghanistan and the World,2021-08-19,"Yet Again, the US Was Defeated in Afghanistan....",NaN,1.0
62390,Biden Visits Saudi Arabia,2022-07-15,Two visitors and the consequences: Biden visit...,NaN,1.0


In [318]:
len(reddit_short2.groupby('title_q').count())

639

In [319]:
NUM_SAMPLE = 5
samples = []
for i, group in reddit_short2.groupby('title_q'):
    group = group.sort_values('upvote_ratio', ascending = False)
    new_group = group.iloc[:NUM_SAMPLE]
    samples.append(new_group)

In [320]:
reddit_short2 = pd.concat(samples, axis = 0)

In [321]:
reddit_short2

,title_q,date_q,title,selftext,upvote_ratio
0,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
1,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
2,Impeachment Hearings Continue,2019-11-22,"Hyper-Partisan Witnesses, Democrats Continue E...",NaN,None
56967,Impeachment Hearings Continue,2019-11-22,"Hyper-Partisan Witnesses, Democrats Continue E...",NaN,None
56968,Impeachment Hearings Continue,2019-11-22,Closed-Door Impeachment Hearings Will Continue...,NaN,None
...,...,...,...,...,...
26218,‘Public Charge’ Rule,2019-08-15,Dear President Trump: Let’s Make A Deal About ...,NaN,None
26233,‘Public Charge’ Rule,2019-08-15,California attorney general vows to fight 'vil...,NaN,None
60473,‘Public Charge’ Rule,2019-08-15,California sues Trump over ‘public charge’ rul...,NaN,None
60472,‘Public Charge’ Rule,2019-08-15,Public Charge Rule Is Trump's Latest Attack on...,NaN,None


In [322]:
miss_titles = set(src_titles).difference(reddit_short2['title_q'])

In [323]:
len(reddit_short2['title_q'].unique())

639

In [324]:
len(miss_titles)

212

In [325]:
miss_queries = [list(query) for query in queries if query[0] in miss_titles]

In [335]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(miss_queries):
    get_posts_by_news(query, OUTPUT_CSV, subreddit = 'worldnews', date_thresh = None, header = WRITE_HEAD, max_posts = 20)
    print(f'Finished query {i} of {len(miss_queries)}', end = '\r')

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [17]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'

In [18]:
posts = pd.read_csv(OUTPUT_CSV)

C:\Users\ndeas\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (15,16,23,24,25,26,27,29,30,36,39,40,42,46,49,51,56,57,58,61,67,73) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
src_titles = both_queries[:, 0]

In [19]:
reddit_short3 = posts[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

In [20]:
len(reddit_short3)

62755

In [21]:
len(reddit_short3['title_q'].unique())

703

In [23]:
miss_titles = set(src_titles).difference(reddit_short3['title_q'])

In [24]:
len(miss_titles)

148

In [37]:
miss_url_queries = [list(query) for query in both_queries if query[0] in miss_titles]

In [38]:
len(miss_url_queries)

148

In [54]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(miss_url_queries):
    title = query[0]
    urls = query[1]
    date = query[2]
    for url in urls:
        print(f'Searching url {url}...', end = '\r')
        one_query = (title, url, date)
        get_posts_by_news(one_query, OUTPUT_CSV, title_or_url = 'url', subreddit = 'politics', header = WRITE_HEAD, max_posts = 10)
    print(f'Finished query {i} of {len(miss_url_queries)}', end = '\r')

Query (' EOs Regarding Transgender Rights', 'https://apnews.com/article/biden-reverse-ban-transgender-military-f0ace4f9866e0ca0df021eba75b3af20', 'January 26, 2021') returned 2 results.
Query (' EOs Regarding Transgender Rights', 'https://apnews.com/article/montana-bills-judiciary-helena-060d274f606d9e8e08b786be0398efd3', 'January 26, 2021') returned 0 results. 
Query (' EOs Regarding Transgender Rights', 'https://www.vox.com/identities/2021/1/25/22248553/trans-military-ban-revoked-biden', 'January 26, 2021') returned 0 results. 
Query (' EOs Regarding Transgender Rights', 'https://slate.com/news-and-politics/2021/01/biden-executive-order-trump-trans-ban.html', 'January 26, 2021') returned 1 results.
Query (' EOs Regarding Transgender Rights', 'https://www.washingtonpost.com/opinions/2021/01/25/transgender-ban-lifted-tubman-20-its-hard-time-conservatives/', 'January 26, 2021') returned 0 results. 
Query (' EOs Regarding Transgender Rights', 'https://slate.com/news-and-politics/2020/06/

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)


Query (' EOs Regarding Transgender Rights', 'https://www.dailysignal.com/2021/01/21/will-biden-place-political-correctness-ahead-of-military-readiness-on-transgender-policy/', 'January 26, 2021') returned 0 results. 
Query (' EOs Regarding Transgender Rights', 'https://www.wsj.com/articles/joe-bidens-first-day-began-the-end-of-girls-sports-11611341066', 'January 26, 2021') returned 3 results.
Query (' EOs Regarding Transgender Rights', 'https://townhall.com/columnists/robjenkins/2021/01/25/the-transgender-agenda-will-eviscerate-title-ix-n2583636', 'January 26, 2021') returned 0 results. 
Query (' EOs Regarding Transgender Rights', 'https://nypost.com/2021/01/22/bidens-trans-order-undoes-decades-of-feminist-progress/', 'January 26, 2021') returned 1 results.
Query (' EOs Regarding Transgender Rights', 'https://www.spectator.co.uk/article/biden-s-trans-rights-agenda-is-bad-news-for-women', 'January 26, 2021') returned 1 results.
Query (' EOs Regarding Transgender Rights', 'https://www.cn

Query (' General Election Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=0bee859e05&e=6f16cb1c23', 'October 28, 2020') returned 0 results. 
Query (' General Election Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=1d4f807ae8&e=6f16cb1c23', 'October 28, 2020') returned 0 results. 
Query (' General Election Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=390ef02489&e=6f16cb1c23', 'October 28, 2020') returned 0 results. 
Query (' General Election Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=e02a267dcf&e=6f16cb1c23', 'October 28, 2020') returned 0 results. 
Query (' General Election Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=7504450594&e=6f16cb1c23', 'October 28, 2020') returned 0 results. 
Query (' General Election Update', 'https://theflipside.us15.list

Query (' NYT Scoop and NYS Bill re Trump’s Taxes', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=88d40d05ed&e=3b34384e6b', 'May 9, 2019') returned 0 results. 
Query (' NYT Scoop and NYS Bill re Trump’s Taxes', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=24abce593f&e=3b34384e6b', 'May 9, 2019') returned 0 results. 
Query (' NYT Scoop and NYS Bill re Trump’s Taxes', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=1b4014eea7&e=3b34384e6b', 'May 9, 2019') returned 0 results. 
Query (' NYT Scoop and NYS Bill re Trump’s Taxes', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=8f694106bc&e=3b34384e6b', 'May 9, 2019') returned 0 results. 
Query (' NYT Scoop and NYS Bill re Trump’s Taxes', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=153e4d63f1&e=3b34384e6b', 'May 9, 2019') returned 0 results. 
Query (' N

...

Query ('Bernie’s Millionaire Status and Town Hall', 'https://www.reuters.com/article/us-usa-election-sanders/bernie-sanders-tax-returns-show-he-became-millionaire-after-2016-white-house-run-idUSKCN1RR2CJ', 'April 17, 2019') returned 0 results. 
Query ('Bernie’s Millionaire Status and Town Hall', 'https://www.foxnews.com/politics/bernie-sanders-town-hall-with-fox-news-live-blog', 'April 17, 2019') returned 0 results. 
Query ('Bernie’s Millionaire Status and Town Hall', 'https://www.foxnews.com/opinion/sen-bernie-sanders-we-need-an-economy-and-government-that-works-for-all-not-just-the-one-percent', 'April 17, 2019') returned 1 results.
Query ('Bernie’s Millionaire Status and Town Hall', 'https://www.washingtonpost.com/opinions/2019/04/10/if-youre-bothered-by-bernies-millionaire-status-vote-him/?noredirect=on&utm_term=.41845aa18984', 'April 17, 2019') returned 0 results. 
Query ('Bernie’s Millionaire Status and Town Hall', 'https://theweek.com/articles/835476/bernies-millionaire-now-does

Query ('Biden 2020 Update', 'https://www.reuters.com/article/us-usa-election-biden-climate/biden-unveils-1-7-trillion-climate-plan-to-end-u-s-carbon-emissions-by-2050-idUSKCN1T515R', 'June 6, 2019') returned 1 results.
Query ('Biden 2020 Update', 'https://www.politico.com/story/2019/06/05/biden-abortion-funding-ban-1509136', 'June 6, 2019') returned 1 results.
Query ('Biden 2020 Update', 'https://www.reuters.com/article/us-usa-election-michigan/biden-sanders-lead-trump-in-battleground-state-michigan-poll-idUSKCN1T61L5', 'June 6, 2019') returned 0 results. 
Query ('Biden 2020 Update', 'https://www.vox.com/2019/6/5/18652787/climate-change-2020-ocasio-cortez-biden-warren', 'June 6, 2019') returned 1 results.
Query ('Biden 2020 Update', 'https://www.theatlantic.com/science/archive/2019/06/bidens-climate-plan-mini-green-new-deal/591046/', 'June 6, 2019') returned 1 results.
Query ('Biden 2020 Update', 'https://www.dailykos.com/stories/2019/6/5/1862849/-Biden-campaign-says-he-supports-the-an

Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=0339886f56&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=3c08c2f3d7&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=5c553dd3c6&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=4f875a1c58&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=4810cadfe0&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?

Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=0339886f56&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=3c08c2f3d7&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Cabinet Update', 'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=5c553dd3c6&e=6f16cb1c23', 'December 8, 2020') returned 0 results. 
Query ('Biden Holds News Conference', 'https://www.thedailyupside.com/?utm_source=The%20Flip%20Side&utm_medium=Newsletter&utm_campaign=Promotion', 'March 26, 2021') returned 0 results. 
Query ('Biden Holds News Conference', 'https://www.reuters.com/article/usa-biden/update-7-biden-compares-xi-to-putin-republican-voting-restriction-plans-to-segregation-era-laws-idUSL1N2LN26K', 'March 26, 2021') returned 0 results. 
Query ('Biden Holds News Conferen

KeyboardInterrupt: 